# COPD Hospitalisations | Processing

The main tasks completed to clean and preprocess this dataset were:

**Data Manipulation**
1. Rename columns.
2. Reformat 'financial year' values from XX/YY to XXXX/YYYY
3. Remove 'LHD' from LHD name values.
4. Remove 'All' data (Representing a state-wide average).
5. Remove seperate age groups, keeping only rows with "All Ages"
6. Remove columns holding Confidence Interval data.
7. Remove rows holding 'Persons' data in the sex column (Representing a genderless rate per 100,000).
8. Remove 'risk group' column.

## Set Up

Ensure that the required libraries are available by running the below code in the terminal before execution:
- pip install pandas


Execute the following in the jupyter notebook before execution to ensure that the required libraries are imported:

In [58]:
import pandas as pd

## Load Dataset

In [59]:
# File path.
file_path = 'data-raw.csv'

# Read the file.
df = pd.read_csv(file_path)

df

,Risk group,Sex,LHD,Period,"Rate per 100,000 population",LL 95% CI,UL 95% CI
0,35-64 years,Males,Sydney LHD,1-Feb,119.6,98.1,144.3
1,35-64 years,Males,Sydney LHD,2-Mar,126.9,104.7,152.2
2,35-64 years,Males,Sydney LHD,3-Apr,127.4,105.3,152.6
3,35-64 years,Males,Sydney LHD,4-May,151.4,127.5,178.4
4,35-64 years,Males,Sydney LHD,5-Jun,125.1,103.9,149.4
...,...,...,...,...,...,...,...
3023,All ages,Persons,All LHDs,17/18,235.5,232.4,238.6
3024,All ages,Persons,All LHDs,18/19,225.8,222.8,228.8
3025,All ages,Persons,All LHDs,19/20,205.1,202.3,207.9
3026,All ages,Persons,All LHDs,20/21,145.6,143.3,148


## Data Manipulation

Rename columns to match Air Quality data set.

In [60]:
# Rename columns.
df = df.rename(columns={
    'LHD': 'lhd',
    'Period': 'financial year'
})

# Set column names to lower case.
df.columns = df.columns.str.lower()

Make Year data in 'financial year' column more verbose.

In [61]:
# Reformat 'financial year' values from XX/YY to XXXX/YYYY.
df['financial year'] = df['financial year'].apply(
    lambda x: f'20{x[:2]}/20{x[3:]}' if isinstance(x, str) else x
)

Remove LHD from Local Area Districts values.

In [62]:
# Remove ' LHD' from the 'lhd' column.
df['lhd'] = df['lhd'].str.replace(' LHD', '')

Remove rows representing state-wide aggregated date.

In [63]:
# Remove rows with NaN in the 'lhd' column.
df = df.dropna(subset=['lhd'])

# Remove rows with 'All' in the 'lhd' column.
df = df[~df['lhd'].str.contains('All')]

Remove rows representing an aggregate of all risk groups within a Local Health District.

In [64]:
# Remove rows with NaN in the 'risk group' column.
df = df.dropna(subset=['risk group'])

# Remove rows with 'All ages' in the 'risk group' column.
df = df[df['risk group'].str.contains('All ages')]

df

,risk group,sex,lhd,financial year,"rate per 100,000 population",ll 95% ci,ul 95% ci
2020,All ages,Males,Sydney,201-/20eb,313.6,289.8,338.8
2021,All ages,Males,Sydney,202-/20ar,282.3,259.4,306.5
2022,All ages,Males,Sydney,203-/20pr,298.8,274.3,324.7
2023,All ages,Males,Sydney,204-/20ay,326.7,301.3,353.5
2024,All ages,Males,Sydney,205-/20un,283.7,261.6,307.2
...,...,...,...,...,...,...,...
3002,All ages,Persons,Far West,2017/2018,486,421,559.3
3003,All ages,Persons,Far West,2018/2019,479.9,414.8,553.4
3004,All ages,Persons,Far West,2019/2020,425.6,365.9,493.5
3005,All ages,Persons,Far West,2020/2021,351,296.1,414.1


Remove rows holding Confidence Interval data.

In [65]:
# Drop columns with '% ci' in the header
df = df.loc[:, ~df.columns.str.contains('% ci')]

Remove rows holding "Persons" data in the sex column.

In [66]:
# Drop rows with 'Persons' in the 'sex' column.
df = df[~df['sex'].str.contains('Persons')]

Remove the "Risk Group" column

In [67]:
# Drop the 'risk group' column.
df = df.drop(columns=['risk group'])

Drop rows where column titles had been entered as data.

In [68]:
# Drop rows where column title had been entered as data.
df = df[~df['sex'].str.contains('Sex')]

## Output Processed Dataset

In [69]:
# File path.
file_path_output = 'data-processed.csv'

# Save the file.
df.to_csv(file_path_output, index=False)

## View Dataset

In [70]:
df

,sex,lhd,financial year,"rate per 100,000 population"
2020,Males,Sydney,201-/20eb,313.6
2021,Males,Sydney,202-/20ar,282.3
2022,Males,Sydney,203-/20pr,298.8
2023,Males,Sydney,204-/20ay,326.7
2024,Males,Sydney,205-/20un,283.7
...,...,...,...,...
2666,Females,Far West,2017/2018,450.9
2667,Females,Far West,2018/2019,408
2668,Females,Far West,2019/2020,413.9
2669,Females,Far West,2020/2021,242.2


## Alternative Approach
Composite primary key becomes only 'lhd' and 'date'.

Tracks 'Male rate per 100,000 population' and 'Female rate per 100,000 population' on the same row.

### Reconfigure Table

In [71]:
# Convert 'rate per 100,000 population' to numeric
df['rate per 100,000 population'] = pd.to_numeric(df['rate per 100,000 population'], errors='coerce')

# Pivot the dataframe to have 'sex' as columns
df_alt = df.pivot_table(index=['financial year', 'lhd'], columns='sex', values='rate per 100,000 population').reset_index()

# Rename the columns to match the desired format
df_alt.columns.name = None
df_alt = df_alt.rename(columns={'Males': 'Male rate per 100,000 population', 'Females': 'Female rate per 100,000 population'})

df_alt

,financial year,lhd,"Female rate per 100,000 population","Male rate per 100,000 population"
0,201-/20eb,Central Coast,235.4,348.2
1,201-/20eb,Far West,241.6,362.2
2,201-/20eb,Hunter New England,210.8,319.9
3,201-/20eb,Illawarra Shoalhaven,190.9,301.5
4,201-/20eb,Mid North Coast,268.1,351.3
...,...,...,...,...
310,20De/20-13,South Western Sydney,208.4,288.8
311,20De/20-13,Southern NSW,269.5,294.0
312,20De/20-13,Sydney,153.6,263.6
313,20De/20-13,Western NSW,338.7,416.4


In [72]:
# File path.
file_path_output_alt = 'data-processed-alt.csv'

# Save the file.
df_alt.to_csv(file_path_output_alt, index=False)